In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [0]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [0]:
neighborhoods_data = newyork_data['features']

In [0]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [0]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [63]:
address = 'Greenpoint, NY'

geolocator = Nominatim(user_agent="add_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of address are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of address are 40.7237134, -73.9509714.


In [67]:
# individual neighboirhoods to see full details of historic venues
user_map = folium.Map(location=[latitude, longitude], zoom_start=14) 
    
user_map

In [0]:
CLIENT_ID = 'R044AKOKAKL2CXFRUIKFP5VNY2Q4SQMTWGBVINCBBE0XOC5H' # your Foursquare ID
CLIENT_SECRET = 'KXIA15ASPHEEIJQWG3KGUHXHBIS3XBJGWRYAG3MXC0P0JDQ1' # your Foursquare Secret
VERSION = '20200408' # Foursquare API version

radius = 500
LIMIT = 200

locale = address
catID = '4deefb944765f83613cdba6e'

In [0]:

url = 'https://api.foursquare.com/v2/venues/search?near={}&categoryId={}&client_id={}&client_secret={}&v={}&radius={}&limit={}'.format(
    locale,
    catID,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    radius, 
    LIMIT)


In [70]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e907acdd03993001b145385'},
 'response': {'confident': False,
  'geocode': {'feature': {'cc': 'US',
    'displayName': 'Greenpoint, NY, United States',
    'geometry': {'bounds': {'ne': {'lat': 40.7396392822,
       'lng': -73.9293212891},
      'sw': {'lat': 40.7139434814, 'lng': -73.9626846313}},
     'center': {'lat': 40.72371, 'lng': -73.95097}},
    'highlightedName': '<b>Greenpoint</b>, <b>NY</b>, United States',
    'id': 'geonameid:5119383',
    'longId': '72057594043047319',
    'matchedName': 'Greenpoint, NY, United States',
    'name': 'Greenpoint',
    'woeType': 22},
   'parents': [],
   'what': '',
   'where': 'greenpoint ny'},
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'primary': True,
      'shortName': 'Park'}],
    'hasPerk': False,
    'id': 

In [0]:
rep2 = results['response']['venues']
rep2_cols = ['Name', 'Latitude', 'Longitude'] 
rep_prepped = pd.DataFrame(columns=rep2_cols)

for data in rep2:
    site_name = site = data['name'] 
    site_lat = lat = data['location']['lat']
    site_lng = lng = data['location']['lng']
    rep_prepped = rep_prepped.append({'Name': site_name,
                                        'Latitude': site_lat,
                                        'Longitude': site_lng,
                                          }, ignore_index=True)

In [72]:
rep_prepped.head()

,Name,Latitude,Longitude
0,Father Jerzy Popiełuszko Square,40.723118,-73.952397
1,Bates Motel,40.727107,-73.947039


In [78]:

# add markers to map
for name, lat, lng in zip(rep_prepped['Name'], rep_prepped['Latitude'], rep_prepped['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(user_map) 

user_map